In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-25 03:33:11--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-06-25 03:33:11 (6.58 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Start Spark Session 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [5]:
# 1. retrieve all the rows where the total_votes count is equal to or greater than 20.
#filtered_df = df.filter("total_votes>=20")
relevant_votes_df = df.filter(df["total_votes"]>=20)
relevant_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   23282003|R2SHXRL6SL1GC9|B010FP0WRU|     974013859|#392: The EP Coll...|           Music|          3|           25|         26|   N|                Y|Up it by a half-s...|A solid collectio...| 2015-08-31|
|         US|   21715314|R2ZC033X86YOY8|B00ZYBH6M0|     408426475|  Live in Tokyo 1975|           Music|          5|    

In [6]:
# 2. Filter all the rows where the total_votes count is equal to or greater than 20 
significant_votes_df = relevant_votes_df.filter(relevant_votes_df["helpful_votes"]/relevant_votes_df["total_votes"]>=0.5)
significant_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   23282003|R2SHXRL6SL1GC9|B010FP0WRU|     974013859|#392: The EP Coll...|           Music|          3|           25|         26|   N|                Y|Up it by a half-s...|A solid collectio...| 2015-08-31|
|         US|   21715314|R2ZC033X86YOY8|B00ZYBH6M0|     408426475|  Live in Tokyo 1975|           Music|          5|    

In [18]:
# 3. Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid)
paid_review_df = significant_votes_df.filter(significant_votes_df["vine"]=='Y')
paid_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45725380| R84VUCDBCI29U|B0020H46LK|     630657414|        Losing Sleep|           Music|          3|           18|         28|   Y|                N|Suitable for Mass...|Do you still get ...| 2009-07-31|
|         US|   53081008| R4V3ICFDTIDIF|B001SMC91M|     609338227|Tell 'Em What You...|           Music|          4|    

In [19]:
# 4. retrieve all the rows where the review was not part of the Vine program (unpaid)
unpaid_review_df = significant_votes_df.filter(significant_votes_df["vine"]=='N')
unpaid_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   23282003|R2SHXRL6SL1GC9|B010FP0WRU|     974013859|#392: The EP Coll...|           Music|          3|           25|         26|   N|                Y|Up it by a half-s...|A solid collectio...| 2015-08-31|
|         US|   21715314|R2ZC033X86YOY8|B00ZYBH6M0|     408426475|  Live in Tokyo 1975|           Music|          5|    

In [24]:
# 5a. Determine the total number of reviews
from pyspark.sql.functions import count
total_reviews = significant_votes_df.count()
total_reviews

105986

In [25]:
# 5b. the number of 5-star reviews
five_starts_df = significant_votes_df.filter(significant_votes_df["star_rating"]==5)
five_star_reviews_count = five_starts_df.count()
five_star_reviews_count

67580

In [20]:
# 5c. Print total count of paid & unpaid reviews
unpaid_review_count = unpaid_review_df.count()
paid_review_count = paid_review_df.count()
print(unpaid_review_count)
print(paid_review_count)

105979
7


In [22]:
#5d. filter paid & unpaid 5-star reviews
five_unpaid_review_df = unpaid_review_df.filter(unpaid_review_df["star_rating"]==5)
five_paid_review_df = paid_review_df.filter(paid_review_df["star_rating"]==5)
# calculate count of paid & unpaid 5-star count
five_unpaid_review_count = five_unpaid_review_df.count()
five_paid_review_count = five_paid_review_df.count()
print(five_unpaid_review_count)
print(five_paid_review_count)

67580
0


In [23]:
#5e. Percentage of 5-star reviews of paid and unpaid types.
five_star_unpaid_percentage = (five_unpaid_review_count/unpaid_review_count)*100
five_star_unpaid_percentage

63.767350135404186

In [26]:
unpaid_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   23282003|R2SHXRL6SL1GC9|B010FP0WRU|     974013859|#392: The EP Coll...|           Music|          3|           25|         26|   N|                Y|Up it by a half-s...|A solid collectio...| 2015-08-31|
|         US|   21715314|R2ZC033X86YOY8|B00ZYBH6M0|     408426475|  Live in Tokyo 1975|           Music|          5|    

In [32]:
paid_reviews_by_rating = paid_review_df.groupby("star_rating").agg(count("review_id")).withColumnRenamed("count(review_id)","review_count")
paid_reviews_by_rating.show()

+-----------+------------+
|star_rating|review_count|
+-----------+------------+
|          3|           3|
|          4|           4|
+-----------+------------+



In [33]:
unpaid_reviews_by_rating = unpaid_review_df.groupby("star_rating").agg(count("review_id")).withColumnRenamed("count(review_id)","review_count")
unpaid_reviews_by_rating.show()

+-----------+------------+
|star_rating|review_count|
+-----------+------------+
|          1|        9157|
|          3|        7897|
|          5|       67580|
|          4|       15997|
|          2|        5348|
+-----------+------------+

